[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the EECS JupyterHub as well as in the binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [128]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [129]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data (the code below will be hidden when the dashboard is rendered by Voila).

In [130]:
filekey="admissionbeds"

In [131]:
def access_json_file():
    
    with open(filekey + ".json", "rt") as INFILE:
        jsondata=json.load(INFILE)
        
    return jsondata


In [132]:

jsondata = access_json_file()


## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and generate a dataframe with some random data

In [133]:
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(rawdata, type):
    datalist=rawdata['data']

    dates=[dictionary["date"] for dictionary in datalist]
    dates.sort()
    
    numberofuniquedates=len(set(dates))
    numberofdates=len(dates)
    aredatesunique=numberofdates==numberofuniquedates
    print("total number of date", numberofdates, "total number of unique dates", numberofuniquedates)
    print(" are dates unique =", aredatesunique)

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    print(startdate, " to ", enddate)

    index=pd.date_range(startdate, enddate, freq="D")
    admissionbedsdf=pd.DataFrame(index=index, columns=[type, "vbo"])


    for entry in datalist:
        date=parse_date(entry["date"])
        for column in [type, "vbo"]:
            if pd.isna(admissionbedsdf.loc[date, column]):
                value= float(entry[column]) if entry [column]!=None else 0.0
                admissionbedsdf.loc[date, column]=value

    admissionbedsdf.fillna(0.0, inplace=True)
    return admissionbedsdf



df=wrangle_data(jsondata, "admissions")

total number of date 185 total number of unique dates 185
 are dates unique = True
2020-01-03 00:00:00  to  2020-07-05 00:00:00


## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting;
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

Clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [134]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    
    filters = ['areaType=overview']

    structure = {
        "date" : "date",
        "admissions" : "newAdmissions",
        "vbo" : "covidOccupiedMVBeds",
        "cases" : "newCasesByPublishDate",
    }


    api = Cov19API(filters=filters, structure=structure)

    admissionbeds=api.get_json()
    return admissionbeds

In [135]:
def refresh_json_file():
    
    data=access_api()
    
    with open(filekey + ".json", "wt") as OUTF:
        json.dump(data, OUTF)
        
#refresh_json_file()


In [136]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    global jsondata
    jsondata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    #df=wrangle_data(apidata, "admissions")
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='PANIC', # you may want to change this...
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Keep calm and carry on",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='exclamation-triangle'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='danger', description='PANIC', icon='exclamation-triangle', style=ButtonStyle(), tooltip='…

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [137]:
def plot_graph(type):
    
    df=wrangle_data(jsondata, type)
    df.plot()

typeselect=wdg.Dropdown(
    options=["admissions", "cases"],
    value="admissions",
    description= "compare",
    disabled=False
)    
    

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=typeselect.value
    if current==typeselect.options[0]:
        other=typeselect.options[1]
    else:
        other=typeselect.options[0]
    typeselect.value=other # forces the redraw
    typeselect.value=current # now we can change it back
    


graph=wdg.interactive_output(plot_graph, {"type" : typeselect}) 

display(typeselect, graph)
        


Dropdown(description='compare', options=('admissions', 'cases'), value='admissions')

Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*